In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
test_data = pd.read_csv('smurf.csv')
test_data.sample(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong,urgent,count,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Unnamed: 26,Unnamed: 27
2044,0,tcp,other,OTH,1514,0,0,0,0,8181,...,97,97,1.0,0.0,0.99,0.0,0,0,0,0
495,0,tcp,other,OTH,1514,0,0,0,0,6689,...,97,97,1.0,0.0,0.99,0.0,0,0,0,0
985,0,tcp,other,OTH,1514,0,0,0,0,7164,...,97,97,1.0,0.0,0.99,0.0,0,0,0,0
620,0,tcp,other,OTH,1514,0,0,0,0,6811,...,97,97,1.0,0.0,0.99,0.0,0,0,0,0
3016,0,tcp,other,OTH,1514,0,0,0,0,9118,...,96,96,1.0,0.0,0.99,0.0,0,0,0,0


In [3]:
test_data.shape

(4095, 28)

In [4]:
list(test_data)

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong',
 'urgent',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate ',
 'diff_srv_rate ',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'Unnamed: 26',
 'Unnamed: 27']

In [7]:
test_data["protocol_type"].value_counts()

tcp     4090
udp        3
icmp       2
Name: protocol_type, dtype: int64

In [12]:
num_feature = ['duration',
 'protocol_type',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong',
 'urgent',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate ',
 'diff_srv_rate ',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

In [13]:
data = test_data[num_feature]

In [14]:
data.dtypes

duration                         int64
protocol_type                   object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong                            int64
urgent                           int64
count                            int64
srv_count                        int64
serror_rate                      int64
srv_serror_rate                  int64
rerror_rate                      int64
diff_srv_rate                    int64
srv_diff_host_rate             float64
dst_host_count                 float64
dst_host_srv_count             float64
dst_host_same_srv_rate           int64
dst_host_diff_srv_rate           int64
dst_host_same_src_port_rate    float64
dst_host_srv_diff_host_rate    float64
dst_host_serror_rate           float64
dst_host_srv_serror_rate       float64
dst_host_rerror_rate             int64
dst_host_srv_rerror_rate         int64
dtype: object

In [15]:
data["protocol_type"].value_counts()

tcp     4090
udp        3
icmp       2
Name: protocol_type, dtype: int64

In [16]:
cleanup_nums = {"protocol_type": {"icmp": 0, "tcp": 1, "udp": 2}}
data.replace(cleanup_nums, inplace=True)
data.head(6)

/Library/Python/3.7/site-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


,duration,protocol_type,src_bytes,dst_bytes,land,wrong,urgent,count,srv_count,serror_rate,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,1514,0,0,0,0,6210,6210,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0
1,0,1,1514,0,0,0,0,6211,6211,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0
2,0,1,1514,0,0,0,0,6212,6212,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0
3,0,1,1514,0,0,0,0,6213,6213,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0
4,0,1,1514,0,0,0,0,6214,6214,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0
5,0,1,1514,0,0,0,0,6215,6215,0,...,0.0,0.0,95,95,1.0,0.0,0.99,0.0,0,0


In [17]:
data.shape

(4095, 24)

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(data)

In [19]:
json_file = open('model_ANN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_ANN.h5")
print("Loaded model from disk")

Loaded model from disk


In [20]:
loaded_model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [21]:
y_pred=loaded_model.predict(X)
y_pred =(y_pred>0.5)

In [24]:
y_pred.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 1])

In [25]:
label = ['neptune', 'normal', 'smurf']
pred = []
for i in y_pred.argmax(axis=1):
    pred.append(label[i])

In [26]:
print(pred)

['neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'normal', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'normal', 'neptune', 'normal', 'normal', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'neptune', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'smurf', 'sm

In [33]:
result = pd.DataFrame(pred, columns=['pred'])

In [34]:
result

,pred
0,neptune
1,neptune
2,neptune
3,neptune
4,neptune
5,neptune
6,neptune
7,neptune
8,neptune
9,neptune
